In [1]:
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('sec_lit_releases_2018_full.csv')

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
df.head()

,Release No.,Date,Respondents
0,LR-24380,"Dec. 26, 2018","Cherubim Interests, Inc., PDX Partners, Inc., Victura Construction Group, Inc., and Patrick Jevon Johnson"
1,LR-24379,"Dec. 21, 2018","John T. Place, et al."
2,LR-24378,"Dec. 21, 2018","China United Insurance Service, Inc., et al. See also: SEC Complaint"
3,LR-24377,"Dec. 20, 2018","Grenda Group, LLC, et al. See also: SEC Complaint"
4,LR-24376,"Dec. 19, 2018","Robert S. ""Lute"" Davis, Jr., et al., Jordan E. Goodman., Alan H. New, et al. See also: Davis Complaint; Goodman Complaint; New Complaint"


In [5]:
# make a new column of lowercase release numbers to generate URL column
df['release_no'] = df['Release No.'].str.replace('-', '')
df['release_no'] = df['release_no'].str.lower()

In [6]:
# add the url column based on the url pattern
df['url'] = "https://www.sec.gov/litigation/litreleases/2018/" + df.release_no + ".htm"

In [7]:
# drop the release_no column
df = df.drop(['release_no'], axis=1)

In [8]:
df

,Release No.,Date,Respondents,url
0,LR-24380,"Dec. 26, 2018","Cherubim Interests, Inc., PDX Partners, Inc., Victura Construction Group, Inc., and Patrick Jevon Johnson",https://www.sec.gov/litigation/litreleases/2018/lr24380.htm
1,LR-24379,"Dec. 21, 2018","John T. Place, et al.",https://www.sec.gov/litigation/litreleases/2018/lr24379.htm
2,LR-24378,"Dec. 21, 2018","China United Insurance Service, Inc., et al. See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2018/lr24378.htm
3,LR-24377,"Dec. 20, 2018","Grenda Group, LLC, et al. See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2018/lr24377.htm
4,LR-24376,"Dec. 19, 2018","Robert S. ""Lute"" Davis, Jr., et al., Jordan E. Goodman., Alan H. New, et al. See also: Davis Complaint; Goodman Complaint; New Complaint",https://www.sec.gov/litigation/litreleases/2018/lr24376.htm
5,LR-24375,"Dec. 17, 2018",Peter Cho See also: SEC Complaint,https://www.sec.gov/litigation/litreleases/2018/lr24375.htm
6,LR-24374,"Dec. 14, 2018","Craig Arsenault, Atlas Capital Management, Inc., and ACT Global Investments See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2018/lr24374.htm
7,LR-24373,"Dec. 14, 2018","Thomas H. Laws, et al. See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2018/lr24373.htm
8,LR-24372,"Dec. 13, 2018","Gregory E. Webb, et al.",https://www.sec.gov/litigation/litreleases/2018/lr24372.htm
9,LR-24371,"Dec. 10, 2018","Rajeshwar R. Gannamaneni, Deepthi Gandra, and Linga R. Gannamaneni See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2018/lr24371.htm


In [9]:
# using BeautifulSoup loop through each url and pull all the text behind it

def get_release_info(row):
    
    try:
        request = requests.get(row['url'])
        soup = BeautifulSoup(request.text, 'html.parser')
        
        full_text = soup.findAll(text=True)
        
    except:
        return pd.Series({})

    return pd.Series({
        'full_text': full_text,
    })

In [10]:
df_litreleases_18 = df.apply(get_release_info, axis=1).join(df)

In [11]:
df_litreleases_18.head()

,full_text,Release No.,Date,Respondents,url
0,"[html, , , , , , , , , , , , , Cherubim Interests, Inc., PDX Partners, Inc., Victura Construction Group, Inc., and Patrick Jevon Johnson (Release No. LR-24380; Dec. 26, 2018), , , , , , , <script type=""text/javascript"" src=""/js/third-party/jquery.tools.min.js""></script> , , , , // adding the following class via JS allows for the creation of JS-specific style rules $('html').addClass('js'); // ForeSee Production Embed Script v2.00 // DO NOT MODIFY BELOW THIS LINE ***************************************** ;(function (g) { var d = document, am = d.createElement('script'), h = d.head || d.getElementsByTagName(""head"")[0], fsr = 'fsReady', aex = { ""src"": ""//gateway.foresee.com/sites/sec-gov/production/gateway.min.js"", ""type"": ""text/javascript"", ""async"": ""true"", ""data-vendor"": ""fs"", ""data-role"": ""gateway"" }; for (var attr in aex){am.setAttribute(attr, aex[attr]);}h.appendChild(am);g[fsr] = function () {var aT = '__' + fsr + '_stk__';g[aT] = g[aT] || [];g[aT].push(arguments);}; })(window); // DO NOT MODIFY ABOVE THIS LINE ***************************************** , , , (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){ (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o), m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m) })(window,document,'script','//www.google-analytics.com/analytics.js','ga'); ga('create', 'UA-30394047-1', 'auto'); ga('set', 'anonymizeIp', true); ga('send', 'pageview'); , , , , , , , , , , , , , , , , , Search SEC Documents, , , , , , , Company Filings, | , More Search Options, , , Skip to Main Content, , , , , About, , , What We Do, , Commissioners, , Securities Laws, , Reports and Publications, , Careers, , Contact, , , , , Divisions & Offices, , , Corporation Finance, , Enforcement, , Investment Management, , Economic and Risk Analysis, , Trading and Markets, , Administrative Law Judges, , Office of Compliance Inspections and Examinations, , Regional Offices, , All Divisions and Offices, , ...]",LR-24380,"Dec. 26, 2018","Cherubim Interests, Inc., PDX Partners, Inc., Victura Construction Group, Inc., and Patrick Jevon Johnson",https://www.sec.gov/litigation/litreleases/2018/lr24380.htm
1,"[html, , , , , , , , , , , , , John T. Place, et al. (Release No. LR-24379; Dec. 21, 2018), , , , , , , <script type=""text/javascript"" src=""/js/third-party/jquery.tools.min.js""></script> , , , , // adding the following class via JS allows for the creation of JS-specific style rules $('html').addClass('js'); // ForeSee Production Embed Script v2.00 // DO NOT MODIFY BELOW THIS LINE ***************************************** ;(function (g) { var d = document, am = d.createElement('script'), h = d.head || d.getElementsByTagName(""head"")[0], fsr = 'fsReady', aex = { ""src"": ""//gateway.foresee.com/sites/sec-gov/production/gateway.min.js"", ""type"": ""text/javascript"", ""async"": ""true"", ""data-vendor"": ""fs"", ""data-role"": ""gateway"" }; for (var attr in aex){am.setAttribute(attr, aex[attr]);}h.appendChild(am);g[fsr] = function () {var aT = '__' + fsr + '_stk__';g[aT] = g[aT] || [];g[aT].push(arguments);}; })(window); // DO NOT MODIFY ABOVE THIS LINE ***************************************** , , , (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){ (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o), m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m) })(window,document,'script','//www.google-analytics.com/analytics.js','ga'); ga('create', 'UA-30394047-1', 'auto'); ga('set', 'anonymizeIp', true); ga('send', 'pageview'); , , , , , , , , , , , , , , , , , Search SEC Documents, , , , , , , Company Filings, | , More Search Options, , , Skip to Main Content, , , , , About, , , What We Do, , Commissioners, , Securities Laws, , Reports and Publications, , Careers, , Contact, , , , , Divisions & Offic

In [12]:
df_litreleases_18.dtypes

full_text      object
Release No.    object
Date           object
Respondents    object
url            object
dtype: object

In [13]:
df_litreleases_18 = df_litreleases_18[['Release No.','Respondents','Date','url', 'full_text']]
df_litreleases_18.head()

,Release No.,Respondents,Date,url,full_text
0,LR-24380,"Cherubim Interests, Inc., PDX Partners, Inc., Victura Construction Group, Inc., and Patrick Jevon Johnson","Dec. 26, 2018",https://www.sec.gov/litigation/litreleases/2018/lr24380.htm,"[html, , , , , , , , , , , , , Cherubim Interests, Inc., PDX Partners, Inc., Victura Construction Group, Inc., and Patrick Jevon Johnson (Release No. LR-24380; Dec. 26, 2018), , , , , , , <script type=""text/javascript"" src=""/js/third-party/jquery.tools.min.js""></script> , , , , // adding the following class via JS allows for the creation of JS-specific style rules $('html').addClass('js'); // ForeSee Production Embed Script v2.00 // DO NOT MODIFY BELOW THIS LINE ***************************************** ;(function (g) { var d = document, am = d.createElement('script'), h = d.head || d.getElementsByTagName(""head"")[0], fsr = 'fsReady', aex = { ""src"": ""//gateway.foresee.com/sites/sec-gov/production/gateway.min.js"", ""type"": ""text/javascript"", ""async"": ""true"", ""data-vendor"": ""fs"", ""data-role"": ""gateway"" }; for (var attr in aex){am.setAttribute(attr, aex[attr]);}h.appendChild(am);g[fsr] = function () {var aT = '__' + fsr + '_stk__';g[aT] = g[aT] || [];g[aT].push(arguments);}; })(window); // DO NOT MODIFY ABOVE THIS LINE ***************************************** , , , (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){ (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o), m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m) })(window,document,'script','//www.google-analytics.com/analytics.js','ga'); ga('create', 'UA-30394047-1', 'auto'); ga('set', 'anonymizeIp', true); ga('send', 'pageview'); , , , , , , , , , , , , , , , , , Search SEC Documents, , , , , , , Company Filings, | , More Search Options, , , Skip to Main Content, , , , , About, , , What We Do, , Commissioners, , Securities Laws, , Reports and Publications, , Careers, , Contact, , , , , Divisions & Offices, , , Corporation Finance, , Enforcement, , Investment Management, , Economic and Risk Analysis, , Trading and Markets, , Administrative Law Judges, , Office of Compliance Inspections and Examinations, , Regional Offices, , All Divisions and Offices, , ...]"
1,LR-24379,"John T. Place, et al.","Dec. 21, 2018",https://www.sec.gov/litigation/litreleases/2018/lr24379.htm,"[html, , , , , , , , , , , , , John T. Place, et al. (Release No. LR-24379; Dec. 21, 2018), , , , , , , <script type=""text/javascript"" src=""/js/third-party/jquery.tools.min.js""></script> , , , , // adding the following class via JS allows for the creation of JS-specific style rules $('html').addClass('js'); // ForeSee Production Embed Script v2.00 // DO NOT MODIFY BELOW THIS LINE ***************************************** ;(function (g) { var d = document, am = d.createElement('script'), h = d.head || d.getElementsByTagName(""head"")[0], fsr = 'fsReady', aex = { ""src"": ""//gateway.foresee.com/sites/sec-gov/production/gateway.min.js"", ""type"": ""text/javascript"", ""async"": ""true"", ""data-vendor"": ""fs"", ""data-role"": ""gateway"" }; for (var attr in aex){am.setAttribute(attr, aex[attr]);}h.appendChild(am);g[fsr] = function () {var aT = '__' + fsr + '_stk__';g[aT] = g[aT] || [];g[aT].push(arguments);}; })(window); // DO NOT MODIFY ABOVE THIS LINE ***************************************** , , , (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){ (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o), m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m) })(window,document,'script','//www.google-analytics.com/analytics.js','ga'); ga('create', 'UA-30394047-1', 'auto'); ga('set', 'anonymizeIp', true); ga('send', 'pageview'); , , , , , , , , , , , , , , , , , Search SEC Documents, , , , , , , Company Filings, | , More Search Options, , , Skip to Main Content, , , , , About, , , What We Do, , 

In [14]:
# Save to CSV
df_litreleases_18.to_csv("litreleases_2018_scraped_full.csv", index=False)